## Expected Free Energy minimization for mobile robot navigation

Wouter Kouw, last update: 06-06-2024

### Dynamics

Consider a mobile robot that moves according to:

$$\underbrace{\begin{bmatrix} x_{1,k} \\ x_{2,k} \\ \dot{x}_{1,k} \\ \dot{x}_{2,k} \end{bmatrix}}_{z_k} = \underbrace{\begin{bmatrix} 1 & 0 & \Delta t & 0 \\ 0 & 1 & 0 & \Delta t \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}}_{A} \underbrace{\begin{bmatrix} x_{1,k-1} \\ x_{2,k-1} \\ \dot{x}_{1,k-1} \\ \dot{x}_{2,k-1} \end{bmatrix}}_{z_{k-1}} + \underbrace{\begin{bmatrix} 0 & 0 \\ 0 & 0 \\ \Delta t & 0 \\ 0 & \Delta t \end{bmatrix}}_{B} \underbrace{\begin{bmatrix} u_{1,k} \\ u_{2,k}\end{bmatrix}}_{u_k} + q_k \, .$$

Process noise is white, $q_k \sim \mathcal{N}(0, Q)$, with 

$$Q = \begin{bmatrix} \frac{\Delta t^3}{3} \rho_1 & 0 & \frac{\Delta t^2}{2} \rho_1 & 0 \\
                      0 & \frac{\Delta t^3}{3} \rho_2 & 0 & \frac{\Delta t^2}{2} \rho_2 \\
                      \frac{\Delta t^2}{2} \rho_1 & 0 & \Delta t \rho_1 & 0 \\
                      0 & \frac{\Delta t^2}{2} \rho_2 & 0 & \Delta t \rho_2 \end{bmatrix} \, .$$

### Observations: polar coordinates from beacon

In the middle of a training area, there is a beacon that reports range and angle to the robot. This will allow it to infer its position in the field through a polar-to-cartesian transformation:

$$\begin{align}
y_{1,k} &= g_1(z_k) = \sqrt{x_{1,k}^2 + x_{2,k}^2} \\
y_{2,k} &= g_2(z_k) = \mathrm{atan}(x_{1,k}, x_{2,k}) \, ,
\end{align}$$

where $\mathrm{atan}$ refers to the arctangent. We denote this as:

$$y_k = g(z_k) + r_k$$

where $r_k$ is measurement noise. 

### Probabilistic model

The above system can be captured with a probabibilistic state-space model of the form:

$$\begin{align}
p(z_0) &= \mathcal{N}(z_0 \mid m_0, S_0) \\
p(z_k \mid z_{k-1}, u_k) &= \mathcal{N}(z_k \mid Az_{k-1} + Bu_k, Q) \\
p(y_k \mid z_k) &= \mathcal{N}(y_k \mid g(z_k), R) \, .
\end{align}$$

### Gaussian approximation to likelihood

The joint will be:
$$\begin{align}
p(y_k, z_k ) &\approx \mathcal{N} \big(\begin{bmatrix} z_k \\ y_k \end{bmatrix} \mid \begin{bmatrix} m_k \\ \mu_k \end{bmatrix}, \begin{bmatrix} S_k & \Gamma_k \\ \Gamma_k^{\top} & \Sigma_k \end{bmatrix} \big) 
\end{align}$$

Because it is a Gaussian, it can be conditioned to:
$$\begin{align}
p(y_k | z_k) &\approx \mathcal{N} \big(y_k | \mu_k + \Gamma_k^{\intercal} S_k^{-1}(x_k - m_k), \Sigma_k - \Gamma_k S_k^{-1} \Gamma_k^{\intercal} \big)
\end{align}$$

### Posterior predictive

$$\begin{align}
p(y_k \mid y_{1:k-1}) &= \int p(y_k \mid z_k) p(z_k \mid y_{1:k-1}) dz_k \\
&= \int \mathcal{N} \big(\begin{bmatrix} z_k \\ y_k \end{bmatrix} \mid \begin{bmatrix} m_k \\ \mu_k \end{bmatrix}, \begin{bmatrix} S_k & \Gamma_k \\ \Gamma_k^{\top} & \Sigma_k \end{bmatrix} \big) dz_k \\
&= \mathcal{N}(y_k \mid \mu_k, \Sigma_k)
\end{align}$$

### Planning and control

EFE Objective for a single time step:

$$\begin{align}
    \mathcal{J}_k(u_t) = \mathrm{C}  +  \frac{1}{2} \text{tr}\big(S_{*}^{-1} ( \Sigma_t + \Sigma_{*} ) \big)  +  \frac{1}{2} \ln \frac{\big|\Sigma_t  -  \Gamma^{\intercal}_t \bar{S}_t^{- 1} \Gamma_t \big|}{\big| \Sigma_t \big|} 
\end{align}$$

Minimizing this objective for a time horizon of $T$ steps:

$$\begin{align}
\hat{u} &= \underset{\bar{u} \in \mathcal{U}}{\arg \max} \ q^{*}(\bar{u}) \\
&= \underset{\bar{u} \in \mathcal{U}}{\arg \min} \sum_{t=1}^{T} \mathcal{J}_k(u_t) - \ln p(u_t)  \, .
\end{align}$$

## Experiments

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using Revise
using Colors
using Optim
using ForwardDiff
using ProgressMeter
using LinearAlgebra
using ControlSystems
using Distributions
using StatsPlots
using Plots
default(label="", grid=false, linewidth=3, markersize=3, margin=15Plots.pt)
includet("../Robots.jl"); using. Robots
includet("../FreeEnergyAgents.jl"); using. FreeEnergyAgents
includet("../util.jl");

In [ ]:
# Time
Δt = 0.2
len_trial = 20
tsteps = range(0, step=Δt, length=len_trial)
len_horizon = 5;

# Nonlinear observation
g(x::AbstractVector) = [sqrt(x[1]^2 + x[2]^2), atan(x[2],x[1])]

# Setpoint (desired observation)
z_star = [0.0, .5, 0.0, 0.0]
goal = (g(z_star), 0.5diagm(ones(2)))

# Parameters
σ = 1e-3
ρ = [1e-2, 1e-2]
η = 0.0

# Limits of controller
u_lims = (-1.0, 1.0)
opts = Optim.Options(time_limit=20)

# Initial state
z_0 = [0.0, -.5, 0., 0.]

# Initial belief
m_0 = z_0
S_0 = 0.5diagm(ones(4));

fbot  = FieldBot(g, ρ, σ=σ, Δt=Δt, control_lims=u_lims)
agent = EFEAgent(goal, g, ρ, σ=σ; η=η, Δt=Δt, time_horizon=len_horizon)

### Model: EFE (ET2)

In [ ]:
# Preallocate
z_est  = (zeros(4,len_trial), zeros(4,4,len_trial))
z_pln  = (zeros(len_trial, 4,len_horizon), zeros(len_trial, 4,4,len_horizon))
y_pln  = (zeros(len_trial, 2,len_horizon), zeros(len_trial, 2,2,len_horizon))
z_sim  = zeros(4,len_trial)
y_sim  = zeros(2,len_trial)
u_sim  = zeros(2,len_trial)
F_EFE2 = zeros(len_trial)
J_EFE2 = zeros(len_trial)

# Initial state
z_sim[:,1] = z_0

# Start recursion
m_kmin1 = m_0
S_kmin1 = S_0

@showprogress for k in 2:len_trial
    
    "Interact with environment"

    # Update system with selected control
    y_sim[:,k], z_sim[:,k] = update(fbot, z_sim[:,k-1], u_sim[:,k-1])
               
    "State estimation"
    
    m_k_pred, S_k_pred = predict(agent, m_kmin1, S_kmin1, u_sim[:,k-1])
    m_k,S_k = correct(agent, y_sim[:,k], m_k_pred, S_k_pred, approx="ET2")
    
    # Compute model evidence
    F_EFE2[k] = evidence(agent, y_sim[:,k], m_k_pred, S_k_pred, approx="ET2")
    J_EFE2[k] = -logpdf(MvNormal(goal[1], goal[2]), y_sim[:,k])
    
    # Store state estimates
    z_est[1][:,k] = m_k
    z_est[2][:,:,k] = S_k
    
    "Planning"
    
    # Single-argument objective
    G(u::AbstractVector) = EFE(agent, u, (m_k,S_k), approx="ET2")
    
    # Call minimizer using constrained L-BFGS procedure
    results = Optim.optimize(G, u_lims[1], u_lims[2], zeros(2*len_horizon), Fminbox(LBFGS()), opts; autodiff=:forward)
    
    # Extract minimizing control
    policy = reshape(Optim.minimizer(results), (2,len_horizon))
    u_sim[:,k] = policy[:,1]

    # Planning
    planned_states, planned_obs = planned_trajectory(agent, policy, (m_k,S_k), approx="ET2")
    z_pln[1][k,:,:]   = planned_states[1]
    z_pln[2][k,:,:,:] = planned_states[2] 
    y_pln[1][k,:,:]   = planned_obs[1]
    y_pln[2][k,:,:,:] = planned_obs[2]
   
    # Update recursion
    m_kmin1 = m_k
    S_kmin1 = S_k
    
end

In [ ]:
xl = [-2., 2.]
yl = [-1., 3.]

anim = @animate for k in 3:len_trial
    p301 = plot(size=(800,800), title="$k/$len_trial", legend=:topleft, aspect_ratio=:equal, ylabel="position y", xlabel="position x", xlims=xl, ylims=yl)
    scatter!([z_0[1]], [z_0[2]], color="blue", label="start", markersize=5)
    scatter!([z_star[1]], [z_star[2]], color="red", label="goal state", markersize=5)
    
    plot!(z_sim[1,2:k], z_sim[2,2:k], c="blue", label="true", alpha=0.5)    
    
    plot!(z_est[1][1,2:k], z_est[1][2,2:k], c="purple", label="inferred", alpha=0.5)
    covellipse!(z_est[1][1:2,k], z_est[2][1:2,1:2,k], n_std=1, color="purple", linewidth=0, fillalpha=0.3)

    plot!(z_pln[1][k,1,:], z_pln[1][k,2,:], c="orange", label="planned")
    for j in 1:len_horizon
        covellipse!(z_pln[1][k,1:2,j], z_pln[2][k,1:2,1:2,j], n_std=1, color="orange", linewidth=0, fillalpha=0.3)
    end
end
gif(anim, "animations/cart2polar-experiment-EFE2-path.gif", fps=3)

In [ ]:

p301 = plot(size=(800,500), legend=:topleft, ylabel="position y", xlabel="position x", xlims=xl, ylims=yl)
scatter!([z_0[1]], [z_0[2]], color="green", marker=:diamond, label="start", markersize=8)
scatter!([z_star[1]], [z_star[2]], color="green", label="goal", markersize=8)

plot!(z_sim[1,:], z_sim[2,:], c="blue", label="true")    
plot!(z_est[1][1,:], z_est[1][2,:], c="purple", label="inferred")

for j in 1:len_trial
    covellipse!(z_est[1][1:2,j], z_est[2][1:2,1:2,j], n_std=1, color="purple", linewidth=0, fillalpha=0.1)
end
plot!()

In [ ]:
savefig("figures/cart2polar-experiment-EFE2-path.png")

### Model: EFE (ET1)

In [ ]:
# Preallocate
z_est  = (zeros(4,len_trial), zeros(4,4,len_trial))
z_pln  = (zeros(len_trial, 4,len_horizon), zeros(len_trial, 4,4,len_horizon))
y_pln  = (zeros(len_trial, 2,len_horizon), zeros(len_trial, 2,2,len_horizon))
z_sim  = zeros(4,len_trial)
y_sim  = zeros(2,len_trial)
u_sim  = zeros(2,len_trial)
F_EFE1 = zeros(len_trial)
J_EFE1 = zeros(len_trial)

# Initial state
z_sim[:,1] = z_0

# Start recursion
m_kmin1 = m_0
S_kmin1 = S_0

# Initialize policy
policy = zeros(2len_horizon)

μ = zeros(2)  
Σ = zeros(2,2)
Γ = zeros(2,4)

@showprogress for k in 2:len_trial
    
    "Interact with environment"

    # Update system with selected control
    y_sim[:,k], z_sim[:,k] = update(fbot, z_sim[:,k-1], u_sim[:,k-1])
               
    "State estimation"
    
    m_k_pred, S_k_pred = predict(agent, m_kmin1, S_kmin1, u_sim[:,k-1])
    m_k,S_k = correct(agent, y_sim[:,k], m_k_pred, S_k_pred, approx="ET1")
    
    # Compute model evidence
    F_EFE1[k] = evidence(agent, y_sim[:,k], m_k, S_k, approx="ET2")
    J_EFE1[k] = -logpdf(MvNormal(goal[1], goal[2]), y_sim[:,k])
    
    # Store state estimates
    z_est[1][:,k] = m_k
    z_est[2][:,:,k] = S_k
    
    "Planning"
    
    # Single-argument objective
    G(u::AbstractVector) = EFE(agent, u, (m_k,S_k), approx="ET1")
    
    # Call minimizer using constrained L-BFGS procedure
    results = Optim.optimize(G, u_lims[1], u_lims[2], zeros(2*len_horizon), Fminbox(LBFGS()), opts; autodiff=:forward)
    
    # Extract minimizing control
    policy = reshape(Optim.minimizer(results), (2,len_horizon))
    u_sim[:,k] = policy[:,1]

    # Planning
    planned_states, planned_obs = planned_trajectory(agent, policy, (m_k,S_k), approx="ET1")
    z_pln[1][k,:,:]   = planned_states[1]
    z_pln[2][k,:,:,:] = planned_states[2] 
    y_pln[1][k,:,:]   = planned_obs[1]
    y_pln[2][k,:,:,:] = planned_obs[2]
   
    # Update recursion
    m_kmin1 = m_k
    S_kmin1 = S_k
    
end

In [ ]:
anim = @animate for k in 3:len_trial
    p301 = plot(size=(800,800), legend=:topleft, aspect_ratio=:equal, ylabel="position y", xlabel="position x", xlims=xl, ylims=yl)
    scatter!([z_0[1]], [z_0[2]], color="blue", label="start", markersize=5)
    scatter!([z_star[1]], [z_star[2]], color="red", label="goal state", markersize=5)
    
    plot!(z_sim[1,2:k], z_sim[2,2:k], c="blue", label="true", alpha=0.5)
    plot!(z_est[1][1,2:k], z_est[1][2,2:k], c="purple", label="inferred", alpha=0.5)
    covellipse!(z_est[1][1:2,k], z_est[2][1:2,1:2,k], n_std=1, color="purple", linewidth=0, fillalpha=0.3)
    
    plot!(z_pln[1][k,1,:], z_pln[1][k,2,:], c="orange", label="planned")
    for j in 1:len_horizon
        covellipse!(z_pln[1][k,1:2,j], z_pln[2][k,1:2,1:2,j], n_std=1, color="orange", linewidth=0, fillalpha=0.3)
    end
end
gif(anim, "animations/cart2polar-experiment-EFE1-path.gif", fps=3) 

In [ ]:
p301 = plot(size=(800,500), legend=:topleft, ylabel="position y", xlabel="position x", xlims=xl, ylims=yl)
scatter!([z_0[1]], [z_0[2]], color="green", marker=:diamond, label="start", markersize=8)
scatter!([z_star[1]], [z_star[2]], color="green", label="goal", markersize=8)

plot!(z_sim[1,:], z_sim[2,:], c="blue", label="true")    
plot!(z_est[1][1,:], z_est[1][2,:], c="purple", label="inferred")

for j in 1:len_trial
    covellipse!(z_est[1][1:2,j], z_est[2][1:2,1:2,j], n_std=1, color="purple", linewidth=0, fillalpha=0.1)
end
plot!()

In [ ]:
savefig("figures/cart2polar-experiment-EFE1-path.png")

### Model: EFER (ET2 risk only)

In [ ]:
# Preallocate
z_est  = (zeros(4,len_trial), zeros(4,4,len_trial))
z_pln  = (zeros(len_trial, 4,len_horizon), zeros(len_trial, 4,4,len_horizon))
y_pln  = (zeros(len_trial, 2,len_horizon), zeros(len_trial, 2,2,len_horizon))
z_sim  = zeros(4,len_trial)
y_sim  = zeros(2,len_trial)
u_sim  = zeros(2,len_trial)
F_EFER = zeros(len_trial)
J_EFER = zeros(len_trial)

# Initial state
z_sim[:,1] = z_0

# Start recursion
m_kmin1 = m_0
S_kmin1 = S_0

# Initialize policy
policy = zeros(2len_horizon)

μ = zeros(2)  
Σ = zeros(2,2)
Γ = zeros(2,4)

@showprogress for k in 2:len_trial
    
    "Interact with environment"

    # Update system with selected control
    y_sim[:,k], z_sim[:,k] = update(fbot, z_sim[:,k-1], u_sim[:,k-1])
               
    "State estimation"
    
    m_k_pred, S_k_pred = predict(agent, m_kmin1, S_kmin1, u_sim[:,k-1])
    m_k,S_k = correct(agent, y_sim[:,k], m_k_pred, S_k_pred, approx="ET2")
    
    # Compute model evidence
    F_EFER[k] = evidence(agent, y_sim[:,k], m_k, S_k, approx="ET2")
    J_EFER[k] = -logpdf(MvNormal(goal[1], goal[2]), y_sim[:,k])
    
    # Store state estimates
    z_est[1][:,k] = m_k
    z_est[2][:,:,k] = S_k
    
    "Planning"
    
    # Single-argument objective
    G(u::AbstractVector) = EFE(agent, u, (m_k,S_k), approx="ET2", add_ambiguity=false)
    
    # Call minimizer using constrained L-BFGS procedure
    results = Optim.optimize(G, u_lims[1], u_lims[2], zeros(2*len_horizon), Fminbox(LBFGS()), opts; autodiff=:forward)
    
    # Extract minimizing control
    policy = reshape(Optim.minimizer(results), (2,len_horizon))
    u_sim[:,k] = policy[:,1]

    # Planning
    planned_states, planned_obs = planned_trajectory(agent, policy, (m_k,S_k), approx="ET2")
    z_pln[1][k,:,:]   = planned_states[1]
    z_pln[2][k,:,:,:] = planned_states[2] 
    y_pln[1][k,:,:]   = planned_obs[1]
    y_pln[2][k,:,:,:] = planned_obs[2]
   
    # Update recursion
    m_kmin1 = m_k
    S_kmin1 = S_k
    
end

In [ ]:
anim = @animate for k in 3:len_trial
    p301 = plot(size=(800,800), legend=:topleft, aspect_ratio=:equal, ylabel="position y", xlabel="position x", xlims=xl, ylims=yl)
    scatter!([z_0[1]], [z_0[2]], color="blue", label="start", markersize=5)
    scatter!([z_star[1]], [z_star[2]], color="red", label="goal state", markersize=5)
    
    plot!(z_sim[1,2:k], z_sim[2,2:k], c="blue", label="true", alpha=0.5)
    plot!(z_est[1][1,2:k], z_est[1][2,2:k], c="purple", label="inferred", alpha=0.5)
    covellipse!(z_est[1][1:2,k], z_est[2][1:2,1:2,k], n_std=1, color="purple", linewidth=0, fillalpha=0.3)
    
    plot!(z_pln[1][k,1,:], z_pln[1][k,2,:], c="orange", label="planned")
    for j in 1:len_horizon
        covellipse!(z_pln[1][k,1:2,j], z_pln[2][k,1:2,1:2,j], n_std=1, color="orange", linewidth=0, fillalpha=0.3)
    end
end
gif(anim, "animations/cart2polar-experiment-EFER-path.gif", fps=3) 

In [ ]:
p301 = plot(size=(800,500), legend=:topleft, ylabel="position y", xlabel="position x", xlims=xl, ylims=yl)
scatter!([z_0[1]], [z_0[2]], color="green", marker=:diamond, label="start", markersize=8)
scatter!([z_star[1]], [z_star[2]], color="green", label="goal", markersize=8)

plot!(z_sim[1,:], z_sim[2,:], c="blue", label="true")    
plot!(z_est[1][1,:], z_est[1][2,:], c="purple", label="inferred")

for j in 1:len_trial
    covellipse!(z_est[1][1:2,j], z_est[2][1:2,1:2,j], n_std=1, color="purple", linewidth=0, fillalpha=0.1)
end
plot!()

In [ ]:
savefig("figures/cart2polar-experiment-EFER-path.png")

### Comparison

In [ ]:
println("F EFE2 = $tF_EFE2")
println("F EFE1 = $tF_EFE1")
println("F EFER = $tF_EFER")

In [ ]:
tJ_EFE2 = sum(J_EFE2)
tJ_EFE1 = sum(J_EFE1)
tJ_EFER = sum(J_EFER)

println("Total goal prior -log-likelihood EFE2 = $tJ_EFE2")
println("Total goal prior -log-likelihood EFE1 = $tJ_EFE1")
println("Total goal prior -log-likelihood EFER = $tJ_EFER")

In [ ]:
perf_EFE2 = sum(F_EFE2) + sum(J_EFE2)
perf_EFE1 = sum(F_EFE1) + sum(J_EFE1)
perf_EFER = sum(F_EFER) + sum(J_EFER)

println("Performance EFE2 = $perf_EFE2")
println("Performance EFE1 = $perf_EFE1")
println("Performance EFER = $perf_EFER")